In [1]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-1-01"
med_start = "2018-10-01"
long_start = "2018-01-01"
short_end = "2019-2-01"
med_end = "2019-2-01"
long_end = "2019-2-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'DASH' , 'EOS' , 'BSV']
long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [2]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')

returns = df.pct_change()
returns = returns.dropna()

In [3]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [4]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-28,-0.015762,-0.008059,-0.041116,-0.015899,-0.011436,-0.027421,-0.012676,-0.009260,-0.012753,-0.040084
2019-01-29,-0.039804,-0.073208,-0.054156,-0.042738,-0.036614,-0.051827,-0.087020,-0.084908,-0.018725,-0.050942
2019-01-30,-0.011546,-0.005519,0.004357,-0.010674,-0.010384,-0.010063,0.006746,0.005468,0.002884,0.000210
2019-01-31,0.033482,0.014351,0.012584,0.031500,0.017678,0.050649,0.005264,0.005294,0.000658,0.030711
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.003123,-0.012632


In [5]:
#drop sectors that don't have data back to 1/1/2018
for i in long_drop:
    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [6]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [7]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-28,-0.015762,-0.008059,-0.041116,-0.015899,-0.011436,-0.027421,-0.012676,-0.009260,-0.012753,-0.040084
2019-01-29,-0.039804,-0.073208,-0.054156,-0.042738,-0.036614,-0.051827,-0.087020,-0.084908,-0.018725,-0.050942
2019-01-30,-0.011546,-0.005519,0.004357,-0.010674,-0.010384,-0.010063,0.006746,0.005468,0.002884,0.000210
2019-01-31,0.033482,0.014351,0.012584,0.031500,0.017678,0.050649,0.005264,0.005294,0.000658,0.030711
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.003123,-0.012632


In [8]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [33]:
data_coins_week

,BTC,ETH,XRP,XLM,LTC,BCH,DASH,EOS,ADA,BSV
2019-01-01 00:00:00,3812.084860,135.215620,0.362950,0.115160,30.929990,152.519780,81.506450,2.645680,0.042380,87.613940
2019-01-02 00:00:00,4005.079300,143.611010,0.375860,0.118980,32.527100,166.905390,83.582780,2.762860,0.044000,94.317060
2019-01-03 00:00:00,3899.180160,155.276080,0.369580,0.115770,32.812310,169.123790,82.914370,2.888200,0.043950,92.872580
2019-01-04 00:00:00,3903.819110,150.597270,0.363430,0.114610,32.176280,161.896680,81.538360,2.742640,0.043460,89.083200
2019-01-05 00:00:00,3893.770900,156.181100,0.360600,0.114450,32.397440,160.497660,80.622260,2.756360,0.043830,87.478650
2019-01-06 00:00:00,3869.314350,159.229410,0.371170,0.119210,36.135090,164.289800,84.214240,2.779410,0.047440,93.375380
2019-01-07 00:00:00,4037.986460,157.219050,0.365090,0.119940,38.960510,164.917580,84.563810,2.895380,0.048660,89.473450
2019-01-08 00:00:00,4041.043610,152.993970,0.366300,0.123990,37.986360,160.952880,83.441100,2.805060,0.048230,88.230820
2019-01-09 00:00:00,4045.365300,151.249880,0.368520,0.123740,39.506530,161.306670,81.547010,2.820770,0.048690,87.801710
2019-01-10 00:00:00,3909.975140,147.656400,0.350370,0.114180,37.362100,152.456060,80.033710,2.814190,0.047890,81.080000


In [9]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-01,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.008539,-0.038546
2019-01-02,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.050627,0.062089
2019-01-03,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,-0.026441,0.081227
2019-01-04,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,0.001190,-0.030132
2019-01-05,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,-0.002574,0.037078
2019-01-06,-0.005280,-0.000239,-0.003138,-0.004878,-0.004645,-0.005656,0.005583,0.009005,-0.006281,0.019518
2019-01-07,0.054819,0.055943,0.049205,0.054757,0.064613,0.047422,0.044577,0.046568,0.043592,-0.012626
2019-01-08,-0.021312,-0.027470,-0.028665,-0.021933,-0.017960,-0.035323,-0.011669,-0.009078,0.000757,-0.026874
2019-01-09,0.000974,0.034699,0.007584,0.003465,0.000643,0.007560,0.051679,0.063853,0.001069,-0.011400
2019-01-10,0.002657,0.024065,0.012198,0.004424,-0.000220,0.020139,0.025131,0.033195,-0.033468,-0.023759


In [10]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-01-01 00:00:00
medium start:  2018-11-09 00:00:00
long start:  2018-11-09 00:00:00
end:  2019-01-31 00:00:00
end:  2019-01-31 00:00:00
end:  2019-01-31 00:00:00


In [31]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-01 00:00:00,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.008539,-0.038546
2019-01-02 00:00:00,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.050627,0.062089
2019-01-03 00:00:00,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,-0.026441,0.081227
2019-01-04 00:00:00,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,0.001190,-0.030132
2019-01-05 00:00:00,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,-0.002574,0.037078


In [12]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                            BTC        ETH      XRP      XLM       LTC  \
2019-01-01 00:00:00  3812.08486  135.21562  0.36295  0.11516  30.92999   

                           BCH      DASH      EOS      ADA       BSV  
2019-01-01 00:00:00  152.51978  81.50645  2.64568  0.04238  87.61394  


,BTC,ETH,XRP,XLM,LTC,BCH,DASH,EOS,ADA,BSV
return,-9.597965,-20.433438,-13.902741,-28.716568,2.666765,-25.031612,-16.590736,-11.300686,-8.659745,-26.858865


In [13]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [14]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-28,-0.015762,-0.008059,-0.041116,-0.015899,-0.011436,-0.027421,-0.012676,-0.009260,-0.012753,-0.040084
2019-01-29,-0.039804,-0.073208,-0.054156,-0.042738,-0.036614,-0.051827,-0.087020,-0.084908,-0.018725,-0.050942
2019-01-30,-0.011546,-0.005519,0.004357,-0.010674,-0.010384,-0.010063,0.006746,0.005468,0.002884,0.000210
2019-01-31,0.033482,0.014351,0.012584,0.031500,0.017678,0.050649,0.005264,0.005294,0.000658,0.030711
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.003123,-0.012632


In [15]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

In [16]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-28 00:00:00,-0.015762,-0.008059,-0.041116,-0.015899,-0.011436,-0.027421,-0.012676,-0.009260,-0.012753,-0.040084
2019-01-29 00:00:00,-0.039804,-0.073208,-0.054156,-0.042738,-0.036614,-0.051827,-0.087020,-0.084908,-0.018725,-0.050942
2019-01-30 00:00:00,-0.011546,-0.005519,0.004357,-0.010674,-0.010384,-0.010063,0.006746,0.005468,0.002884,0.000210
2019-01-31 00:00:00,0.033482,0.014351,0.012584,0.031500,0.017678,0.050649,0.005264,0.005294,0.000658,0.030711
2019-02-01 00:00:00,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.003123,-0.012632


In [17]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [43]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2018-11-09 00:00:00,-0.026167,-0.029946,-0.034528,-0.026649,-0.018112,-0.041334,-0.028960,-0.027914,-0.010130,-0.027927
2018-11-10 00:00:00,-0.011391,-0.014572,-0.025633,-0.011984,-0.014761,-0.009042,-0.017091,-0.016594,-0.003145,-0.004076
2018-11-11 00:00:00,0.004543,0.001072,0.005644,0.004338,0.003019,0.006728,0.001380,0.001450,-0.001590,0.012265
2018-11-12 00:00:00,0.003613,-0.003546,-0.013742,0.002678,0.003084,0.003670,-0.012086,-0.012041,0.000488,-0.004735
2018-11-13 00:00:00,-0.003409,0.004856,-0.012297,-0.003084,-0.007059,0.006228,-0.011437,-0.013355,-0.006087,-0.004134


In [34]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [35]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-28 00:00:00,-0.015762,-0.008059,-0.041116,-0.015899,-0.011436,-0.027421,-0.012676,-0.009260,-0.012753,-0.040084
2019-01-29 00:00:00,-0.039804,-0.073208,-0.054156,-0.042738,-0.036614,-0.051827,-0.087020,-0.084908,-0.018725,-0.050942
2019-01-30 00:00:00,-0.011546,-0.005519,0.004357,-0.010674,-0.010384,-0.010063,0.006746,0.005468,0.002884,0.000210
2019-01-31 00:00:00,0.033482,0.014351,0.012584,0.031500,0.017678,0.050649,0.005264,0.005294,0.000658,0.030711
2019-02-01 00:00:00,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.003123,-0.012632


In [36]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,3812.084860,135.215620
last price,374.603942,313.478539,49.263316,156.644587,20.150812,13.096591,14.062167,43.673755,3446.202290,107.586420
annualized volatility,63.755028,81.209001,76.623222,64.873322,57.900438,83.392298,85.934321,90.411604,39.672379,80.754292
daily volatility,3.337091,4.250673,4.010643,3.395625,3.030647,4.364952,4.498008,4.732360,2.076547,4.226873
return,-0.109171,-0.072171,-0.098472,-0.106369,-0.093820,-0.146661,0.067841,0.093910,-0.095980,-0.204334
Sharpe,-2.595616,-1.381047,-2.017506,-2.495142,-2.510032,-2.560153,0.746088,0.965308,-2.948779,-3.371563
Sortino,-1.311377,0.096531,-0.803105,-1.182303,-0.883263,-1.834711,3.792692,4.161043,0.000000,-3.104593


In [37]:
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,740.953740,637.900186,110.287135,311.303644,40.569745,27.086623,24.329500,73.888661,6490.943350,216.696190
last price,374.603942,313.478539,49.263316,156.644587,20.150812,13.096591,14.062167,43.673755,3446.202290,107.586420
annualized volatility,89.846698,104.944870,109.082910,91.049459,87.665942,104.652156,108.586043,110.590712,70.164137,101.845160
daily volatility,4.702791,5.493066,5.709660,4.765747,4.588645,5.477744,5.683653,5.788582,3.672559,5.330819
return,-0.494430,-0.508577,-0.553318,-0.496811,-0.503304,-0.516492,-0.422012,-0.408925,-0.469075,-0.503515
Sharpe,-2.798631,-2.367892,-2.608584,-2.771495,-2.974644,-2.449588,-1.615900,-1.481716,-3.508993,-2.438227
Sortino,-0.154084,-0.050134,-0.801259,-0.176185,-0.485139,-0.209864,1.515150,1.742533,0.000000,-0.046227


In [38]:
summary['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
open price,740.953740,637.900186,110.287135,311.303644,73.888661,6490.943350,216.696190
last price,374.603942,313.478539,49.263316,156.644587,43.673755,3446.202290,107.586420
annualized volatility,89.846698,104.944870,109.082910,91.049459,110.590712,70.164137,101.845160
daily volatility,4.702791,5.493066,5.709660,4.765747,5.788582,3.672559,5.330819
return,-0.494430,-0.508577,-0.553318,-0.496811,-0.408925,-0.469075,-0.503515
Sharpe,-2.798631,-2.367892,-2.608584,-2.771495,-1.481716,-3.508993,-2.438227
Sortino,-0.154084,-0.050134,-0.801259,-0.176185,1.742533,0.000000,-0.046227


In [39]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.903592,0.917155,0.994501,0.995601,0.980205,0.830279,0.824413,0.243402,0.832111
twenty,0.903592,1.000000,0.890762,0.932918,0.900660,0.935484,0.960411,0.960411,0.272727,0.783358
forty,0.917155,0.890762,1.000000,0.929619,0.913856,0.934384,0.855938,0.837977,0.313416,0.803886
total,0.994501,0.932918,0.929619,1.000000,0.987903,0.987170,0.871701,0.863636,0.247801,0.843842
currency,0.995601,0.900660,0.913856,0.987903,1.000000,0.972507,0.818548,0.815249,0.262830,0.815616
platform,0.980205,0.935484,0.934384,0.987170,0.972507,1.000000,0.864736,0.854472,0.189516,0.833944
application,0.830279,0.960411,0.855938,0.871701,0.818548,0.864736,1.000000,0.990469,0.287390,0.769062
ERC20,0.824413,0.960411,0.837977,0.863636,0.815249,0.854472,0.990469,1.000000,0.271261,0.746334
BTC,0.243402,0.272727,0.313416,0.247801,0.262830,0.189516,0.287390,0.271261,1.000000,0.243768
ETH,0.832111,0.783358,0.803886,0.843842,0.815616,0.833944,0.769062,0.746334,0.243768,1.000000


In [40]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.940571,0.956322,0.998339,0.988411,0.966602,0.904202,0.897870,0.718917,0.854094
twenty,0.940571,1.000000,0.946453,0.952980,0.930936,0.944655,0.968556,0.970823,0.672601,0.848270
forty,0.956322,0.946453,1.000000,0.963690,0.943991,0.948759,0.926559,0.918272,0.720559,0.863025
total,0.998339,0.952980,0.963690,1.000000,0.986691,0.971370,0.918566,0.912742,0.716592,0.860621
currency,0.988411,0.930936,0.943991,0.986691,1.000000,0.929334,0.903713,0.896013,0.752277,0.801348
platform,0.966602,0.944655,0.948759,0.971370,0.929334,1.000000,0.895114,0.888997,0.624057,0.902072
application,0.904202,0.968556,0.926559,0.918566,0.903713,0.895114,1.000000,0.995388,0.680965,0.797850
ERC20,0.897870,0.970823,0.918272,0.912742,0.896013,0.888997,0.995388,1.000000,0.671722,0.797186
BTC,0.718917,0.672601,0.720559,0.716592,0.752277,0.624057,0.680965,0.671722,1.000000,0.574692
ETH,0.854094,0.848270,0.863025,0.860621,0.801348,0.902072,0.797850,0.797186,0.574692,1.000000


In [41]:
corr['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
ten,1.000000,0.940571,0.956322,0.998339,0.897870,0.718917,0.854094
twenty,0.940571,1.000000,0.946453,0.952980,0.970823,0.672601,0.848270
forty,0.956322,0.946453,1.000000,0.963690,0.918272,0.720559,0.863025
total,0.998339,0.952980,0.963690,1.000000,0.912742,0.716592,0.860621
ERC20,0.897870,0.970823,0.918272,0.912742,1.000000,0.671722,0.797186
BTC,0.718917,0.672601,0.720559,0.716592,0.671722,1.000000,0.574692
ETH,0.854094,0.848270,0.863025,0.860621,0.797186,0.574692,1.000000


In [27]:
beta_all

,MTD,QTD,YTD
ten,0.983485,0.987074,0.987074
twenty,1.171971,1.115815,1.115815
forty,1.098066,1.156218,1.156218
total,1.000000,1.000000,1.000000
currency,0.884257,0.955104,NaN
platform,1.265379,1.118776,NaN
application,1.134255,1.109494,NaN
ERC20,1.173415,1.124148,1.124148
BTC,0.338358,0.648627,0.648627
ETH,1.084425,0.984171,0.984171


In [28]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,DASH,EOS,ADA,BSV
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,-0.003033,-0.006457,-0.004450,-0.010418,0.001598,-0.008457,-0.005500,-0.003188,-0.002501,-0.009362
std,0.021086,0.042579,0.027840,0.029939,0.039465,0.040171,0.026019,0.036559,0.029374,0.037477
min,-0.065491,-0.132538,-0.053344,-0.077259,-0.103451,-0.121966,-0.085937,-0.135190,-0.070578,-0.077173
25%,-0.008941,-0.025316,-0.016674,-0.023815,-0.021265,-0.029207,-0.019425,-0.018923,-0.014286,-0.023653
50%,0.000658,-0.008353,-0.005996,-0.006052,0.005463,-0.008641,-0.006475,-0.000324,-0.002764,-0.013888
75%,0.002987,0.013722,0.004687,0.004127,0.015651,0.011445,0.014717,0.018749,0.009535,0.002539
max,0.050627,0.092645,0.101243,0.041590,0.115369,0.094320,0.044553,0.052417,0.082364,0.082634


In [29]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,-0.004534,-0.003073,-0.004235,-0.004435,-0.003982,-0.005849,0.001757,0.002391,-0.003205,-0.007459
std,0.033371,0.042507,0.040106,0.033956,0.030306,0.043650,0.044980,0.047324,0.020765,0.042269
min,-0.107289,-0.117485,-0.123649,-0.108470,-0.097628,-0.137573,-0.107700,-0.107073,-0.065491,-0.132538
25%,-0.017150,-0.029768,-0.030217,-0.018829,-0.013067,-0.032048,-0.020253,-0.024048,-0.009285,-0.027688
50%,-0.003990,-0.001117,0.000228,-0.002864,-0.002941,-0.003465,0.005424,0.005381,0.000166,-0.008654
75%,0.009862,0.024235,0.019846,0.010984,0.009834,0.021384,0.026464,0.031958,0.002936,0.013114
max,0.056704,0.098231,0.069027,0.059887,0.064613,0.083362,0.122128,0.134318,0.050627,0.092645


In [30]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,DASH,EOS,ADA,BSV
return,-9.597965,-20.433438,-13.902741,-28.716568,2.666765,-25.031612,-16.590736,-11.300686,-8.659745,-26.858865
